## Hierarchical attention network for document classification

In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from src.utils import get_max_lengths, get_evaluation
from src.dataset import MyDataset
from src.hierarchical_att_model import HierAttNet
from tensorboardX import SummaryWriter
import shutil
import numpy as np

### Params

In [2]:
class opts:
    def __init__(self):
        self.batch_size = 128
        self.num_epoches = 100
        self.lr = 0.1
        self.momentum = 0.9
        self.word_hidden_size = 50
        self.sent_hidden_size = 50
        self.es_min_delta = 0.0
        self.es_patience = 5
        self.train_set = "data\\TextClassificationDatasets\\ag_news_csv\\train.csv"
        self.test_set = "data\\TextClassificationDatasets\\ag_news_csv\\test.csv"
        self.test_interval = 1
        self.word2vec_path = "data\\glove.6B.50d.txt"
        self.log_path = "tensorboard\\han_voc"
        self.saved_path = "trained_models"
opt = opts()

### Training model

Setup

In [3]:
# Set random seed for reproducibility
if torch.cuda.is_available():
    torch.cuda.manual_seed(123)
else:
    torch.manual_seed(123)

# Create a log file to save training logs
output_file = open(opt.saved_path + os.sep + "logs.txt", "w")
output_file.write("Model's parameters: {}".format(vars(opt)))

# Define DataLoader parameters for training and testing
training_params = {"batch_size": opt.batch_size,
                    "shuffle": True,
                    "drop_last": True}
test_params = {"batch_size": opt.batch_size,
                "shuffle": False,
                "drop_last": False}

In [4]:

# Get maximum sentence and word lengths from training set
max_word_length, max_sent_length = get_max_lengths(opt.train_set)
training_set = MyDataset(opt.train_set, opt.word2vec_path, max_sent_length, max_word_length)
training_generator = DataLoader(training_set, **training_params)
test_set = MyDataset(opt.test_set, opt.word2vec_path, max_sent_length, max_word_length)
test_generator = DataLoader(test_set, **test_params)

In [5]:
# Create the model
model = HierAttNet(opt.word_hidden_size, opt.sent_hidden_size, opt.batch_size, training_set.num_classes,
                    opt.word2vec_path, max_sent_length, max_word_length)

In [6]:

# Create a tensorboard writer to log training process
if os.path.isdir(opt.log_path):
    shutil.rmtree(opt.log_path)
os.makedirs(opt.log_path)
writer = SummaryWriter(opt.log_path)
# writer.add_graph(model, torch.zeros(opt.batch_size, max_sent_length, max_word_length))

if torch.cuda.is_available():
    model.cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=opt.lr, momentum=opt.momentum)
best_loss = 1e5
best_epoch = 0


Training process

In [7]:
model.train()
num_iter_per_epoch = len(training_generator)
for epoch in range(opt.num_epoches):
    # Training phase
    for iter, (feature, label) in enumerate(training_generator):
        if torch.cuda.is_available():
            feature = feature.cuda()
            label = label.cuda()
        optimizer.zero_grad()
        model._init_hidden_state()
        predictions = model(feature)
        loss = criterion(predictions, label)
        loss.backward()
        optimizer.step()
        training_metrics = get_evaluation(label.cpu().numpy(), predictions.cpu().detach().numpy(), list_metrics=["accuracy"])
        
        # Log training information
        print("Epoch: {}/{}, Iteration: {}/{}, Lr: {}, Loss: {}, Accuracy: {}".format(
            epoch + 1,
            opt.num_epoches,
            iter + 1,
            num_iter_per_epoch,
            optimizer.param_groups[0]['lr'],
            loss, training_metrics["accuracy"]))
        writer.add_scalar('Train/Loss', loss, epoch * num_iter_per_epoch + iter)
        writer.add_scalar('Train/Accuracy', training_metrics["accuracy"], epoch * num_iter_per_epoch + iter)
    
    # Testing phase
    if epoch % opt.test_interval == 0:
        model.eval()
        loss_ls = []
        te_label_ls = []
        te_pred_ls = []

        # Testing process
        for te_feature, te_label in test_generator:
            num_sample = len(te_label)
            if torch.cuda.is_available():
                te_feature = te_feature.cuda()
                te_label = te_label.cuda()
            # Forward pass
            with torch.no_grad():
                model._init_hidden_state(num_sample)
                te_predictions = model(te_feature)

            # Compute loss
            te_loss = criterion(te_predictions, te_label)
            loss_ls.append(te_loss * num_sample)
            te_label_ls.extend(te_label.clone().cpu())
            te_pred_ls.append(te_predictions.clone().cpu())

        # Compute testing metrics
        te_loss = sum(loss_ls) / test_set.__len__()
        te_pred = torch.cat(te_pred_ls, 0)
        te_label = np.array(te_label_ls)
        test_metrics = get_evaluation(te_label, te_pred.numpy(), list_metrics=["accuracy", "confusion_matrix"])
        
        # Log testing information
        output_file.write(
            "Epoch: {}/{} \nTest loss: {} Test accuracy: {} \nTest confusion matrix: \n{}\n\n".format(
                epoch + 1, opt.num_epoches,
                te_loss,
                test_metrics["accuracy"],
                test_metrics["confusion_matrix"]))
        print("Epoch: {}/{}, Lr: {}, Loss: {}, Accuracy: {}".format(
            epoch + 1,
            opt.num_epoches,
            optimizer.param_groups[0]['lr'],
            te_loss, test_metrics["accuracy"]))
        writer.add_scalar('Test/Loss', te_loss, epoch)
        writer.add_scalar('Test/Accuracy', test_metrics["accuracy"], epoch)
        model.train()
        if te_loss + opt.es_min_delta < best_loss:
            best_loss = te_loss
            best_epoch = epoch
            torch.save(model, opt.saved_path + os.sep + "whole_model_han")

        # Early stopping
        if epoch - best_epoch > opt.es_patience > 0:
            print("Stop training at epoch {}. The lowest loss achieved is {}".format(epoch, te_loss))
            break

e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\word_att_model.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)
e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\sent_att_model.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)


Epoch: 1/100, Iteration: 1/937, Lr: 0.1, Loss: 1.3807331323623657, Accuracy: 0.265625


e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\word_att_model.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)
e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\sent_att_model.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)


Epoch: 1/100, Iteration: 2/937, Lr: 0.1, Loss: 1.3837823867797852, Accuracy: 0.171875


e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\word_att_model.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)
e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\sent_att_model.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)


...

e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\word_att_model.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)
e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\sent_att_model.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)


Epoch: 1/100, Iteration: 613/937, Lr: 0.1, Loss: 0.48320111632347107, Accuracy: 0.859375


e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\word_att_model.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)
e:\OneDrive - Hanoi University of Science and Technology\Code project\ML\NLP\uvipen_HAN\src\sent_att_model.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(output)


Epoch: 1/100, Iteration: 614/937, Lr: 0.1, Loss: 0.269226610660553, Accuracy: 0.90625


KeyboardInterrupt: 